In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from datetime import datetime
from ultralytics import YOLO
import requests
import json

# Load the YOLOv8 model
model = YOLO("C:\\Users\\KimoStore\\ImageProcessing\\.venv\\runs\\detect\\runs\\detect\\train28\\weights\\best.pt")

# Open the external webcam (usually index 0 or 1 for external webcams)
cap = cv2.VideoCapture(1)

# Check if the webcam opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Store the track history
track_history = defaultdict(lambda: [])

# Dictionary to store the initial detection time of each track ID
initial_detection_time = {}

# Open a file to save tracking data
output_file = "tracking_data.csv"
with open(output_file, 'w') as f:
    f.write("Track_ID,Location_X,Location_Y,Time\n")  # Write header

    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the webcam
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True, conf=0.5)

            # Check if there are any detection results
            if results[0].boxes is not None:
                # Get the boxes and track IDs
                boxes = results[0].boxes.xywh.cpu().numpy()
                track_ids = results[0].boxes.id.int().cpu().tolist() if results[0].boxes.id is not None else []

                # Visualize the results on the frame
                annotated_frame = results[0].plot()

                # Plot the tracks
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]

                    # Get the current time
                    current_time = datetime.now()

                    # Calculate the elapsed time since the initial detection
                    if track_id not in initial_detection_time:
                        initial_detection_time[track_id] = current_time
                    elapsed_time = (current_time - initial_detection_time[track_id]).total_seconds()

                    # Append the center point (x, y) and timestamp to the track history
                    track.append((float(x), float(y), elapsed_time))  # x, y center point with timestamp
                    if len(track) > 30:  # retain 30 tracks for 30 frames
                        track.pop(0)

                    # Draw the tracking lines
                    points = np.array([(pt[0], pt[1]) for pt in track], np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

                    # Write to the file
                    f.write(f"{track_id},{x:.2f},{y:.2f},{elapsed_time}\n")

                    # Send the data as a POST request to the server
                    payload = {
                        "track_id": int(track_id),  # Ensure track_id is an int
                        "x": float(x),              # Ensure x is a float
                        "y": float(y),              # Ensure y is a float
                        "timestamp": float(elapsed_time)  # Ensure timestamp is a float
                    }
                    try:
                        response = requests.post("http://localhost:3000/track", headers={"Content-Type": "application/json"}, data=json.dumps(payload))
                        if response.status_code != 200:
                            print(f"Error sending data: {response.text}")
                    except Exception as e:
                        print(f"Exception occurred: {e}")

                # Display the annotated frame
                cv2.imshow("YOLOv8 Tracking", annotated_frame)
            else:
                # No detections in this frame
                cv2.imshow("YOLOv8 Tracking", frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

print(f"Tracking data saved to {output_file}")
